In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm 
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
p = .1
df_main = pd.read_csv('spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and random.random() > p)
df_main

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Bright Side,HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,The Len Price 3,If You Live Round Here,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Cocktail Slippers,In The City,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Lissie,Little Lovin',HARD ROCK 2010
...,...,...,...,...
1288931,2302bf9c64dc63d88a750215ed187f2c,Pearl Jam,Jeremy,iPhone
1288932,2302bf9c64dc63d88a750215ed187f2c,Led Zeppelin,Rock & Roll,iPhone
1288933,2302bf9c64dc63d88a750215ed187f2c,Alice In Chains,Rooster,iPhone
1288934,2302bf9c64dc63d88a750215ed187f2c,Van Halen,Summer Nights,iPhone


In [3]:
df_main.columns = df_main.columns.str.replace('"', '')
df_main.columns = df_main.columns.str.replace('name', '')
df_main.columns = df_main.columns.str.replace(' ', '')
df_main.columns

Index(['user_id', 'artist', 'track', 'playlist'], dtype='object')

In [4]:
df_main = df_main.groupby('artist').filter(lambda x : len(x)>=50)

In [5]:
df_main = df_main[df_main.groupby('user_id').artist.transform('nunique') >= 10]

In [6]:
size = lambda x: len(x)
df_freq = df_main.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

,user_id,artist,freq
366936,defced0ece4ce946160b0d2698142eac,Vitamin String Quartet,327
65119,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,324
108791,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,263
2902,014e695cc6df96011b90a5beb3206012,Ilaiyaraaja,221
293072,b1d4116e7cf150ae7d77413620f5f571,Wolfgang Amadeus Mozart,219


In [7]:
df_artist = pd.DataFrame(df_freq["artist"].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
df_artist.head()

,artist_id,artist
0,0,Vitamin String Quartet
1,1,Ilaiyaraaja
2,2,Wolfgang Amadeus Mozart
3,3,Grateful Dead
4,4,Lata Mangeshkar


In [8]:
df_freq  = pd.merge(df_freq , df_artist, how='inner', on='artist')
df_freq.head()

,user_id,artist,freq,artist_id
0,defced0ece4ce946160b0d2698142eac,Vitamin String Quartet,327,0
1,26b51e580277e131f87e4c7ee4c0887a,Vitamin String Quartet,324,0
2,414050deadb38aafd8d4ad22ca634055,Vitamin String Quartet,263,0
3,5a7c1187380dfd62b4667511333ba4c5,Vitamin String Quartet,37,0
4,e78e1e7b93c32bc27bf458f6cb8a5554,Vitamin String Quartet,33,0


In [9]:
df_pivot = df_freq.pivot(index='user_id',columns='artist_id',values='freq')
df_pivot

artist_id,0,1,2,3,4,5,6,7,8,9,...,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084
user_id,,,,,,,,,,,,,,,,,,,,,
00055176fea33f6e027cd3302289378b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000c11a16c89aa4b14b328080f5954ee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00123e0f544dee3ab006aa7f1e5725a7,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00139e9cb50fb309549e1561b476226d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0019363a0d57e94d39988c31eeb8d015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffc9e5f9869491983a67f254ab6e61a6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffe11226cdea81a2db9262c0ec7f5d71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff60baf392613ed33f745b89a9b38f7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_pivot = df_pivot.fillna(0)
df_pivot

artist_id,0,1,2,3,4,5,6,7,8,9,...,4075,4076,4077,4078,4079,4080,4081,4082,4083,4084
user_id,,,,,,,,,,,,,,,,,,,,,
00055176fea33f6e027cd3302289378b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c11a16c89aa4b14b328080f5954ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00123e0f544dee3ab006aa7f1e5725a7,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00139e9cb50fb309549e1561b476226d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0019363a0d57e94d39988c31eeb8d015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffc9e5f9869491983a67f254ab6e61a6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ffe11226cdea81a2db9262c0ec7f5d71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fff60baf392613ed33f745b89a9b38f7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors = 50,metric = 'cosine')
Model = knn.fit(df_pivot)

In [109]:
def KNN_recommend(index):
    user_id = df_pivot.index[index]
    user = df_pivot.iloc[index,]
    distances, indices = Model.kneighbors([user])
    neighbors=[]
    for item in indices[0][1:]:
        try:
            neighbors.append(df_main[df_main.index == item]["user_id"].values[0])
        except IndexError:
            continue
    neighbors = list(set(neighbors))
    df_neighbor = pd.DataFrame(columns=['user_id', 'artist','freq','artist_id'])
    for item in neighbors:
        df_neighbor = df_neighbor.append(df_freq[df_freq.user_id == item],ignore_index = True)
    df_neighbor = df_neighbor.sort_values('freq',ascending=False)
    df_neighbor_subset = pd.DataFrame({'artist':df_neighbor['artist'], 'freq':df_neighbor['freq']})
    df_neighbor_subset = df_neighbor_subset.groupby('artist').sum()
    df_neighbor_subset.sort_values('freq',ascending=False, inplace = True)
    df_neighbor_subset = df_neighbor_subset.reset_index()
    user_playlists = []
    for artist in df_freq.loc[df_freq.user_id == user_id]['artist']:
        user_playlists.extend(df_main.loc[df_main['artist'] == artist].playlist.squeeze().unique())
    target_artists = []
    for playlist in neighbor_playlists[:10]:
        target_artists.extend(df_main.loc[df_main['playlist'] == playlist].artist.squeeze())
    target_artists = list(set(target_artists))
    size = lambda x: len(x)
    df_playlist_count = df_main.groupby(['playlist']).agg('size').reset_index().rename(columns={0:'freq'})[['playlist', 'freq']].sort_values(['freq'], ascending=False)
    df_playlist_count = df_playlist_count.reset_index(drop = True)
    top_100 = df_playlist_count[0:len(df_playlist_count)//3].sample(n = 100)
    y_artist = df_main[df_main['playlist'].isin(top_100["playlist"])].artist.squeeze().unique()
    return(len(list(set(y_artist) & set(df_neighbor1.artist[0:10]))) * 10) # Accuracy percentage

In [19]:
df_neighbor_subset[0:10]

,artist,freq
0,Johnny Cash,111
1,Red Hot Chili Peppers,44
2,JAY Z,38
3,2Pac,36
4,Blur,32
5,Lana Del Rey,29
6,Aerosmith,28
7,Beyoncé,22
8,AC/DC,22
9,Rihanna,21


# Model Evaluation

In [110]:
accuracy = []
for index in range(100):
    accuracy.append(KNN_recommend(index))
print(sum(accuracy) / 100)

68.4


In [ ]:
#df_freq.loc[df_freq.user_id == '00055176fea33f6e027cd3302289378b']